In [ ]:
import sys
type(sys.path)
for path in sys.path:
    print(path)
sys.path.append('/home/bla390/doppyo/doppyo')
for path in sys.path:
    print(path)

In [ ]:
import numpy as np
import pandas as pd
import xarray as xr
import diagnostic as dgn
from scipy import interpolate
import matplotlib.pyplot as plt
from cartopy import crs as ccrs
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER

In [ ]:
#== READ ==#
filepath = r'/home/bla390/control_run/atmos_daily_1500_01_01.nc.isobaric'
dataVars = xr.open_dataset(filepath, chunks={'time': 1})

index = list(np.arange(0,59)) + list(np.arange(334,365))
hght_DJF = dataVars.hght.isel(time=index).mean("time")
hght_MAM = dataVars.hght.isel(time=slice( 59, 151)).mean("time")
hght_JJA = dataVars.hght.isel(time=slice(151, 243)).mean("time")
hght_SON = dataVars.hght.isel(time=slice(243, 334)).mean("time")

hght = xr.concat([hght_DJF, hght_MAM, hght_JJA, hght_SON], pd.Index(['DJF', 'MAM', 'JJA', 'SON'], name='season'))
seasonDict = {0:'DJF', 1:'MAM', 2:'JJA', 3:'SON'}

In [ ]:
#== REGRID ==#
new_lon = np.linspace(0, 360, 361) ; new_lat = np.linspace(-89.5,89.5,180)
gh = xr.DataArray(np.zeros((((4,2,180,361)))), coords=[('season', ['DJF', 'MAM', 'JJA', 'SON']), 
                                                      ('level',[300,700]), ('lat',new_lat), ('lon',new_lon)])

for i in np.arange(0, 4):
    
    #--700mb
    ghT = np.transpose(hght.sel(season = seasonDict[i], level=700).values)
    ghf = interpolate.interp2d(dataVars.lon.values,dataVars.lat.values,ghT.ravel())
    gh700_1deg = ghf(new_lon,new_lat)

    #--300mb
    ghT = np.transpose(hght.sel(season = seasonDict[i], level=300).values)
    ghf = interpolate.interp2d(dataVars.lon.values,dataVars.lat.values,ghT.ravel())
    gh300_1deg = ghf(new_lon,new_lat)

    gh_1deg = np.stack((gh300_1deg, gh700_1deg), axis=0)
    gh[i] = xr.DataArray(gh_1deg, coords=[('level',[300,700]), ('lat',new_lat), ('lon',new_lon)])

In [ ]:
#== COMPUTE ==#
ThermalWindMag = xr.DataArray(np.zeros((((4,180,361)))), coords=[('season', ['DJF', 'MAM', 'JJA', 'SON']), 
                                                      ('lat',np.linspace(89.5,-89.5,180)), ('lon',new_lon)])

for i in np.arange(0, 4):
    
    ThermalWind = dgn.thermal_wind(gh.sel(season=seasonDict[i]), plevel_lower=300, plevel_upper=700)
    ThermalWindMag[i] = np.sqrt(np.square(ThermalWind['u_tw']) + np.square(ThermalWind['v_tw']))

In [ ]:
#== PLOT ==#
proj = ccrs.PlateCarree(180)
levels = [0,0.5,1,1.5,2,2.5,3,3.5,4,4.5,5]
fig1 = plt.figure(figsize=(10, 15));
gs1 = plt.GridSpec(4, 1)
gs1.update(left=0.06, right=0.96, bottom = 0.28, top = 0.97, wspace=0.03, hspace = 0.25)
ax1 = plt.subplot(gs1[0, 0], projection=proj)
ax2 = plt.subplot(gs1[1, 0], projection=proj)
ax3 = plt.subplot(gs1[2, 0], projection=proj)
ax4 = plt.subplot(gs1[3, 0], projection=proj)

#--Fig 1
im = ThermalWindMag[0].sel(lat=slice(-2,-89)).plot.contourf(ax=ax1, levels=levels, cmap=plt.cm.GnBu, extend='both',
                                                            add_colorbar = False, transform=ccrs.PlateCarree())
ax1.coastlines()
gl = ax1.gridlines(crs=ccrs.PlateCarree(), draw_labels=True)
gl.xlabels_top = False ; gl.ylabels_right = False
gl.xformatter = LONGITUDE_FORMATTER ; gl.yformatter = LATITUDE_FORMATTER
ax1.set_title('Thermal Wind - ' + seasonDict[0])

#--Fig 2
im = ThermalWindMag[1].sel(lat=slice(-2,-89)).plot.contourf(ax=ax2, levels=levels, cmap=plt.cm.GnBu, extend='both', 
                                                            add_colorbar = False, transform=ccrs.PlateCarree())
ax2.coastlines()
gl = ax2.gridlines(crs=ccrs.PlateCarree(), draw_labels=True)
gl.xlabels_top = False ; gl.ylabels_right = False
gl.xformatter = LONGITUDE_FORMATTER ; gl.yformatter = LATITUDE_FORMATTER
ax2.set_title('Thermal Wind - ' + seasonDict[1])

#--Fig 3
im = ThermalWindMag[2].sel(lat=slice(-2,-89)).plot.contourf(ax=ax3, levels=levels, cmap=plt.cm.GnBu, extend='both', 
                                                            add_colorbar = False, transform=ccrs.PlateCarree())
ax3.coastlines()
gl = ax3.gridlines(crs=ccrs.PlateCarree(), draw_labels=True)
gl.xlabels_top = False ; gl.ylabels_right = False
gl.xformatter = LONGITUDE_FORMATTER ; gl.yformatter = LATITUDE_FORMATTER
ax3.set_title('Thermal Wind - ' + seasonDict[2])

#--Fig 4
im = ThermalWindMag[3].sel(lat=slice(-2,-89)).plot.contourf(ax=ax4, levels=levels, cmap=plt.cm.GnBu, extend='both', 
                                                            add_colorbar = False, transform=ccrs.PlateCarree())
ax4.coastlines()
gl = ax4.gridlines(crs=ccrs.PlateCarree(), draw_labels=True)
gl.xlabels_top = False ; gl.ylabels_right = False
gl.xformatter = LONGITUDE_FORMATTER ; gl.yformatter = LATITUDE_FORMATTER
ax4.set_title('Thermal Wind - ' + seasonDict[3])

#--place colorbar
gs2 = plt.GridSpec(1,1)
gs2.update(left=0.3, right=0.7, bottom = 0.23, top = 0.24)
ax5 = plt.subplot(gs2[0, 0])
plt.colorbar(im, cax=ax5, orientation='horizontal')
#--save image
plt.savefig('/home/bla390/control_run/images/ThermalWind_AllSeasons.png')